In [1]:
import sys,os
path=os.path.dirname(os.getcwd())
sys.path.append(path)
from unityagents  import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
env = UnityEnvironment(file_name=path+'/Tennis_Windows_x86_64/Tennis.exe')
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)
# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)
# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [2]:
from MADDPG_agent import *
from MADDPG_train import *

In [3]:
agent=MADDPG_agent(env,brain_name,buffer_size=50000,batch_size=200,update_every=2,learn_num=4,state_size=24, 
             hidden_in_actor=256,hidden_out_actor=256,out_actor=2,merge=False,full_state_size=48,full_action_size=4,
             hidden_in_critic=256,hidden_out_critic1=256,hidden_out_critic2=128,gauss=True,seed=70,lr_actor=0.0001,
             lr_critic=0.0005,discount_factor=0.99,tau=0.001)

In [ ]:
mean_scores,min_scores,max_scores,moving_avgs,stats=MADDPG_train(agent,env,brain_name,pretraining=True,action_size=2,
                                                                 ratio=0.5,hard_end_noise=True,hard_end=300,noise=5.0,
                                                                 noise_end=0.1,n_episodes=20000,max_t=2000,target_score=0.5,
                                                                 window_size=100,print_every=100,train_mode=True,
                                                                 alter_factor=1.0)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(max_scores)), max_scores, label='network_name')
plt.plot(np.arange(len(moving_avgs)), moving_avgs, c='r', label='average')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend(loc='upper left')
plt.show()

In [ ]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    obs = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        action=agent.act(obs,noise=0.)       # select an action
        env_info = env.step(np.array(action).astype(int))[brain_name]            # send actions to environment
        next_obs = env_info.vector_observations          # get next state
        reward = env_info.rewards                          # get reward
        done = env_info.local_done                         # see if episode has finished        
        obs = next_obs
        scores += reward
        if np.any(done):                                   # exit loop when episode ends
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))